In [10]:
# Cell 1 — Basic setup: paths & imports

import os
from pathlib import Path

import cv2
import matplotlib.pyplot as plt

# Make plots appear inline
%matplotlib inline

# Path to your dataset root (adjust if needed)
DATA_ROOT = Path("../.. /mug_coco_yolo")

TRAIN_IMAGES_DIR = DATA_ROOT / "images" / "train2017"
TRAIN_LABELS_DIR = DATA_ROOT / "labels" / "train2017"

VAL_IMAGES_DIR = DATA_ROOT / "images" / "val2017"
VAL_LABELS_DIR = DATA_ROOT / "labels" / "val2017"

print("Train images dir:", TRAIN_IMAGES_DIR.resolve())
print("Train labels dir:", TRAIN_LABELS_DIR.resolve())
print("Val images dir:", VAL_IMAGES_DIR.resolve())
print("Val labels dir:", VAL_LABELS_DIR.resolve())

print("Train images count:", len(list(TRAIN_IMAGES_DIR.glob("*.jpg"))))
print("Train labels count:", len(list(TRAIN_LABELS_DIR.glob("*.txt"))))


Train images dir: D:\an4sem1\PRS\smart-object-detection\mug_coco_yolo\images\train2017
Train labels dir: D:\an4sem1\PRS\smart-object-detection\mug_coco_yolo\labels\train2017
Val images dir: D:\an4sem1\PRS\smart-object-detection\mug_coco_yolo\images\val2017
Val labels dir: D:\an4sem1\PRS\smart-object-detection\mug_coco_yolo\labels\val2017
Train images count: 0
Train labels count: 0


In [11]:
# Cell 2 — Helper: load YOLO labels for an image

def load_yolo_labels(label_path):
    """
    Load YOLO-format labels from a .txt file.
    Returns a list of (class_id, xc, yc, w, h) as floats.
    """
    boxes = []
    if not label_path.exists():
        return boxes

    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls, xc, yc, w, h = parts
            boxes.append((
                int(cls),
                float(xc),
                float(yc),
                float(w),
                float(h),
            ))
    return boxes

def yolo_to_xyxy(box, img_w, img_h):
    """
    Convert YOLO (xc, yc, w, h) normalized to (x1, y1, x2, y2) absolute pixels.
    """
    cls, xc, yc, w, h = box
    xc_abs = xc * img_w
    yc_abs = yc * img_h
    w_abs = w * img_w
    h_abs = h * img_h

    x1 = int(xc_abs - w_abs / 2)
    y1 = int(yc_abs - h_abs / 2)
    x2 = int(xc_abs + w_abs / 2)
    y2 = int(yc_abs + h_abs / 2)
    return cls, x1, y1, x2, y2


In [12]:
# Cell 3 — Helper: visualize one image with boxes

CLASS_NAMES = {0: "mug"}  # single-class dataset

def show_image_with_boxes(img_path, labels_dir, figsize=(8, 6)):
    img_path = Path(img_path)
    label_path = labels_dir / (img_path.stem + ".txt")

    # Load image with OpenCV (BGR) then convert to RGB for matplotlib
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"Could not read image: {img_path}")
        return
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_h, img_w = img_rgb.shape[:2]

    boxes = load_yolo_labels(label_path)

    # Draw bounding boxes
    img_draw = img_rgb.copy()
    for box in boxes:
        cls, x1, y1, x2, y2 = yolo_to_xyxy(box, img_w, img_h)
        # draw rectangle
        cv2.rectangle(img_draw, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = CLASS_NAMES.get(cls, str(cls))
        # put class text
        cv2.putText(
            img_draw,
            label,
            (x1, max(y1 - 5, 0)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 255, 0),
            2,
            cv2.LINE_AA,
        )

    plt.figure(figsize=figsize)
    plt.imshow(img_draw)
    plt.axis("off")
    plt.title(f"{img_path.name}  |  {len(boxes)} mug(s)")
    plt.show()


In [14]:
# Cell 4 — View a few random training images + labels

import random

train_images = list(TRAIN_IMAGES_DIR.glob("*.jpg"))
print("Total training images found:", len(train_images))

# Show N random examples
N = 5
sample_images = random.sample(train_images, min(N, len(train_images)))

for img_path in sample_images:
    show_image_with_boxes(img_path, TRAIN_LABELS_DIR)


Total training images found: 0


In [5]:
# Cell 5 — View a few random validation images + labels

val_images = list(VAL_IMAGES_DIR.glob("*.jpg"))
print("Total validation images found:", len(val_images))

N = 5
sample_images = random.sample(val_images, min(N, len(val_images)))

for img_path in sample_images:
    show_image_with_boxes(img_path, VAL_LABELS_DIR)


Total validation images found: 0


In [6]:
# Cell 6 — Quick check: any images without labels? (optional sanity check)

def count_images_without_labels(images_dir, labels_dir):
    images = list(images_dir.glob("*.jpg"))
    no_label = []
    for img_path in images:
        label_path = labels_dir / (img_path.stem + ".txt")
        if not label_path.exists():
            no_label.append(img_path.name)
    return no_label

no_label_train = count_images_without_labels(TRAIN_IMAGES_DIR, TRAIN_LABELS_DIR)
no_label_val = count_images_without_labels(VAL_IMAGES_DIR, VAL_LABELS_DIR)

print(f"Train images without labels: {len(no_label_train)}")
print(f"Val images without labels:   {len(no_label_val)}")

# If you want to see a few of them:
print("Some train images without labels:", no_label_train[:10])
print("Some val images without labels:", no_label_val[:10])


Train images without labels: 0
Val images without labels:   0
Some train images without labels: []
Some val images without labels: []


In [7]:
# Cell 7 — (Optional) Show a grid of mugs

from math import ceil

def show_grid(images, labels_dir, rows=2, cols=3, figsize=(12, 8)):
    plt.figure(figsize=figsize)
    for i, img_path in enumerate(images[:rows * cols]):
        img_path = Path(img_path)
        label_path = labels_dir / (img_path.stem + ".txt")

        img = cv2.imread(str(img_path))
        if img is None:
            continue
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_h, img_w = img_rgb.shape[:2]
        boxes = load_yolo_labels(label_path)

        img_draw = img_rgb.copy()
        for box in boxes:
            cls, x1, y1, x2, y2 = yolo_to_xyxy(box, img_w, img_h)
            cv2.rectangle(img_draw, (x1, y1), (x2, y2), (0, 255, 0), 2)

        plt.subplot(rows, cols, i + 1)
        plt.imshow(img_draw)
        plt.axis("off")
        plt.title(img_path.name)

    plt.tight_layout()
    plt.show()

# Example usage with training images
sample_images = random.sample(train_images, min(6, len(train_images)))
show_grid(sample_images, TRAIN_LABELS_DIR, rows=2, cols=3)


<Figure size 1200x800 with 0 Axes>